In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from helper import read_x_train, fill_nan, trans2trader
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [8]:
y_train = pd.read_csv('data/AMF_train_Y.csv')

In [9]:
x_train = read_x_train('data/AMF_train_X.csv', includeShare=True, includeDay=True)
x_train = fill_nan(x_train)

n_rows, n_cols = x_train.shape

### The NaN ratio in each columns

In [12]:
x_train.isnull().sum()/n_rows

Share                         0.0
Day                           0.0
Trader                        0.0
OTR                           0.0
OCR                           0.0
OMR                           0.0
min_time_two_events           0.0
mean_time_two_events          0.0
10_p_time_two_events          0.0
med_time_two_events           0.0
25_p_time_two_events          0.0
75_p_time_two_events          0.0
90_p_time_two_events          0.0
max_time_two_events           0.0
min_lifetime_cancel           0.0
mean_lifetime_cancel          0.0
10_p_lifetime_cancel          0.0
med_lifetime_cancel           0.0
25_p_lifetime_cancel          0.0
75_p_lifetime_cancel          0.0
90_p_lifetime_cancel          0.0
max_lifetime_cancel           0.0
NbTradeVenueMic               0.0
MaxNbTradesBySecond           0.0
MeanNbTradesBySecond          0.0
min_dt_TV1                    0.0
mean_dt_TV1                   0.0
med_dt_TV1                    0.0
min_dt_TV1_TV2                0.0
mean_dt_TV1_TV

In [13]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105782 entries, 0 to 105781
Data columns (total 38 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Share                       105782 non-null  int64  
 1   Day                         105782 non-null  int64  
 2   Trader                      105782 non-null  object 
 3   OTR                         105782 non-null  float64
 4   OCR                         105782 non-null  float64
 5   OMR                         105782 non-null  float64
 6   min_time_two_events         105782 non-null  float64
 7   mean_time_two_events        105782 non-null  float64
 8   10_p_time_two_events        105782 non-null  float64
 9   med_time_two_events         105782 non-null  float64
 10  25_p_time_two_events        105782 non-null  float64
 11  75_p_time_two_events        105782 non-null  float64
 12  90_p_time_two_events        105782 non-null  float64
 13  max_time_two_e

In [14]:
x_train_data = np.array(x_train.drop(['Trader'], axis = 1))

In [15]:
ind2type = {0: 'HFT', 1: 'MIX', 2: 'NON HFT'}
type2ind = {'HFT': 0, 'MIX': 1, 'NON HFT': 2}

In [16]:
y_train

,Trader,type
0,Trader_285,MIX
1,Trader_114,NON HFT
2,Trader_110,NON HFT
3,Trader_57,NON HFT
4,Trader_128,NON HFT
...,...,...
81,Trader_140,MIX
82,Trader_451,MIX
83,Trader_438,MIX
84,Trader_278,HFT


In [17]:
name2type = dict(zip(y_train.Trader, y_train.type))

In [18]:
x_train = x_train.fillna(-1)

In [19]:
x_train['label'] = x_train.apply(lambda row: 0 if name2type[row.Trader] == 'HFT' else (1 if name2type[row.Trader] == 'MIX' else 2), axis=1)

In [80]:
y_train_data = x_train['label']
y_trader_label= x_train[['Trader', 'label']]

In [82]:
# x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(x_train_data, y_train_data, test_size=0.3)

x_train_b_, x_test_b_, y_train_b, y_test_b = train_test_split(x_train.drop('label', axis=1), y_train_data, test_size=0.3)
x_train_b = x_train_b_.drop('Trader', axis=1)
x_test_b = x_test_b_.drop('Trader', axis=1)

In [82]:
x_train_data.shape

In [84]:

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42, n_jobs=-1)
# Train the model on training data
rf.fit(x_train_b, y_train_b)
y_test_pred = rf.predict(x_test_b)

In [119]:
real_type.value_counts()

1    51483
0    31950
2    22349
Name: label, dtype: int64

In [86]:
print(len(np.where(y_test_pred == y_test_b)[0]) / len(y_test_b))
print(metrics.f1_score(
        y_test_b, y_test_pred, average="micro"))

0.9800535686150937
0.9800535686150937


In [105]:
y_trader_label.value_counts()

Trader      label
Trader_209  1        10208
Trader_161  0         7587
Trader_386  0         5808
Trader_430  1         5044
Trader_431  1         2552
                     ...  
Trader_375  2            4
Trader_149  2            4
Trader_191  2            4
Trader_257  2            2
Trader_425  2            1
Length: 86, dtype: int64

In [121]:
final = trans2trader(x_test_b_, y_test_pred)

In [128]:
# traders_in_res = y_trader_label.Trader.unique()
real_type =y_trader_label.drop_duplicates()
# pred_type = pd.DataFrame.from_dict(data=final, orient='index', columns=['type'])['type'].map(type2ind)
pred_type = real_type.Trader.map(final).map(type2ind)
# pred_type = y_trader_label.Trader.map(final).map(type2ind).values

In [130]:
Counter(pred_type)

Counter({2.0: 48, 1.0: 23, 0.0: 13, nan: 1, nan: 1})

In [132]:
print(metrics.f1_score(
        real_type.label, pred_type.fillna(0), average="micro"))

0.9418604651162791


In [30]:
final = {}
for k,v in dicts.items():
    if len(np.where(np.array(v) == 0)[0])/len(v) > 0.85:
        final[k] = 'HFT'
    elif len(np.where(np.array(v) == 1)[0])/len(v) > 0.5:
        final[k] = 'MIX'
    else:
        final[k] = 'NON HFT'

In [133]:
x_real_test = read_x_train('data/AMF_test_X.csv', includeShare=True, includeDay=True)
x_real_test = fill_nan(x_real_test)



In [135]:
x_real_test_data = np.array(x_real_test.drop('Trader', axis=1))

In [136]:
# x_real_test_data['label'] = x_real_test_data.apply(lambda row: 0 if name2type[row.Trader] == 'HFT' else (1 if name2type[row.Trader] == 'MIX' else 2), axis=1)

In [137]:
y_test_predict = rf.predict(x_real_test_data)

In [138]:
x_real_test_data.shape

(85304, 37)

In [139]:
y_test_predict

array([2, 2, 2, ..., 1, 1, 1], dtype=int64)

In [141]:
final = trans2trader(x_real_test, y_test_predict)

In [142]:
from collections import Counter, defaultdict
ratio_dict={}
for trader in dicts:
    _c = dict(Counter(dicts[trader]))
    _sum = sum(_c.values())
    _defdict = defaultdict(lambda: 0)
    for _type in _c:
        _defdict[_type] = _c[_type] / _sum
    ratio_dict[trader] = _defdict


In [145]:
res_df = pd.DataFrame.from_dict(data=final, orient='index', columns=['type'])
res_df['Trader'] = res_df.index
res_df[['Trader', 'type']].set_index('Trader').to_csv('dict_file.csv')